<a href="https://colab.research.google.com/github/settiredd/battleship/blob/main/LayerId.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib.image import imread
import matplotlib.pyplot as plt
import pandas as pd

Mounted at /content/drive


In [ ]:
## Code to convert to HSV
# First written 2/26/2024

# function to convert to HSV
def showImg(imgUrl):
  # read image using url
  origImg = imread(imgUrl)

  # convert to HSV color space
  hsvImg = cv2.cvtColor(origImg, cv2.COLOR_BGR2HSV)

  # access value channel in HSV
  vChan = hsvImg[:,:,2]

  # apply intensity factor to value channel and replace value channel in HSV image
  intensityFactor = 1.25 # can play with the intensity factor to see what best fits creating contrast
  newVChan = np.clip(vChan * intensityFactor, 0, 255).astype(np.uint8)
  hsvImg[:,:,2] = newVChan


  # show image
  fig = plt.figure(1)
  output1 = fig.add_subplot(121)
  output2 = fig.add_subplot(122)
  output1.imshow(origImg)
  output2.imshow(hsvImg[:,:,2], cmap = "gray")

  return hsvImg

In [ ]:
testImg3 = showImg("/content/drive/My Drive/VIP LPRC: Layer Histology/development/A2_Sub.png")

In [ ]:
def enhanceImg(imgURL):
    # using cv2, image loaded is BGR type
    origImg = cv2.imread(imgURL)

    hsvImg = cv2.cvtColor(origImg,cv2.COLOR_BGR2HSV)
    h, s, v = hsvImg[:,:,0], hsvImg[:,:,1], hsvImg[:,:,2]
    v = v.flatten()

    # Calculate histogram for the value channel
    # hist_v = cv2.calcHist([v], [0], None, [256], [0, 256])

    # I think plt.plot is the best option. Can't get plt.hist to work as expected :( the webpages I looked at usually use plt.plot
    # plt.plot(hist_v, color='r', label="value")

    # rint(type(v))

    # Generate random data for the histogram
    # data = np.random.randn(1000)


    plt.hist(v, bins = 10)


    plt.legend(["Frequency of Value"])
    plt.xticks(range(0, 256, 25))
    plt.show()

    # Flatten the V channel and create a DataFrame
    array = v.flatten()
    dataframe = pd.DataFrame(array, columns=['a'])

    # Print the value counts
    print("Value Counts of V Channel:")
    print(dataframe['a'].value_counts(sort = False))

    # returns the image converted to hsv
    return hsvImg, dataframe

In [ ]:
# using original image. This histogram shows the values for the ENTIRE image
original_image, df= enhanceImg("/content/drive/My Drive/VIP LPRC: Layer Histology/development/A2_Sub.png")
# the histogram shows that the picture is mostly white, which I think makes sense

In [ ]:
def imgHists(img):

    # channels = The number of color channels. Ignore this value :) it will be 3 as HSV has 3 channels
    height, width, channels = img.shape

    # hardcoding the tile_size for now
    tile_size = 30

    histograms = []

    # most iterate through height and then width. not sure if it matters
    for y in range(0, height, tile_size):
      for x in range(0, width, tile_size):
        tile = img[y:y+tile_size, x:x+tile_size]

        # since enhanceImg() returns the entirety of the image, we have to do tile[:,:,2] to avoid the weird issue of
        # the histogram dropping off at a specific value. Before, I think it was combining all values of HSV instead of taking just V
        histogram = cv2.calcHist([tile[:,:,2]], [0], None, [256], [0, 256])
        histograms.append(histogram)
    return histograms

In [ ]:
originalImage_hist = imgHists(original_image)

#shows the number of subtiles of the image
print(len(originalImage_hist))

# here, we can print out an individual histogram to see what that tile looks like
plt.plot(originalImage_hist[100])
plt.legend(["Frequency of Value"])
plt.show()

In [ ]:
# need to transform histograms to dataframes (we will cluster the rows)

def toDataFrame(histograms):
  return pd.DataFrame([hist.ravel() for hist in histograms])


In [ ]:
image_df = toDataFrame(originalImage_hist)

In [ ]:
image_df

In [ ]:
image_array = image_df.to_numpy()
image_array

In [ ]:
# helpful documentation: https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering
# for HDBSCAN: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN

import numpy as np
from scipy.spatial.distance import squareform, pdist
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as hac

from sklearn.cluster import AgglomerativeClustering

z = hac.linkage(image_array, metric='cosine', method='complete')
labels = hac.fcluster(z, 0.90, criterion="distance")



# linkage_data = linkage(image_array, method='ward', metric='euclidean')
# dendrogram(linkage_data)

# remove x axis labels
plt.xticks([])
plt.show()


# clustering = AgglomerativeClustering().fit(image_array)
# AgglomerativeClustering()



# labels = clustering.labels_


print(len(set(labels)))

In [ ]:
def imgSegment(img):

    # channels = The number of color channels. Ignore this value :) it will be 3 as HSV has 3 channels
    height, width, channels = img.shape

    # hardcoding the tile_size for now
    tile_size = 30

    histograms = []
    label_num = 0

    # most iterate through height and then width. not sure if it matters
    for y in range(0, height, tile_size):
      for x in range(0, width, tile_size):
        tile = img[y:y+tile_size, x:x+tile_size]
        value = labels[label_num]
        if value == 1:
           color = (0, 0, 255)
        elif value == 2:
          color = (255, 0, 0)  # Yellow if value is 1, blue if 0
        elif value == 3:
          color = (0, 255, 0)
        else:
          color = (0, 255, 255)
        label_num += 1

        # Color the tile
        img[y:y+tile_size, x:x+tile_size] = color

    return img

image = testImg3  # Load your image

# Process the image
result_image = imgSegment(image)

fig = plt.figure(1)
output1 = fig.add_subplot(121)

output1.imshow(result_image)

# Display or save the result_image
#cv2.imshow('Result Image', result_image)
#cv2.waitKey(0)